In [11]:
import pandas as pd
import numpy as np
from os import path

In [3]:
DATA_PATH = '/Users/neal/Desktop/kaggle-experiments/bnp/data'

TRAIN = path.join(DATA_PATH, 'train.csv')
TARGET_CLASS = 'target'

data_set = pd.read_csv(TRAIN)

X = data_set.drop(TARGET_CLASS, axis=1)
y = data_set[TARGET_CLASS]

print 'Shape', X.shape, y.shape
print 'Positive Instances', float(len(y[y == 1]))/len(y)
print 'Negative Instances', float(len(y[y == 0]))/len(y)


Shape (114321, 132) (114321,)
Positive Instances 0.761198729892
Negative Instances 0.238801270108


In [26]:
for f in X.columns:
    num_nans = float(len(X[f]) - X[f].count())
    print 'Feature:', f
    print '\tType:', X[f].dtype
    print '\tSparse:', num_nans / len(X[f])
    if X[f].dtype == 'object':
        values = set(pd.unique(X[f].values))
        if len(values) < 50:
            print '\tValues:', len(values), sorted(values)
        else:
            print '\tValues:', len(values), 'Too many values to print.'
    else:
        print '\tRange:', round(X[f].min(), 2), round(X[f].max(), 2)
        print '\tMean:', round(X[f].mean(), 4), round(X[f].std(), 4)

Feature: ID
	Type: int64
	Sparse: 0.0
	Range: 3.0 228713.0
	Mean: 114228.9282 65934.4874
Feature: v1
	Type: float64
	Sparse: 0.43589541729
	Range: -0.0 20.0
	Mean: 1.6307 1.0828
Feature: v2
	Type: float64
	Sparse: 0.435580514516
	Range: -0.0 20.0
	Mean: 7.4644 2.9617
Feature: v3
	Type: object
	Sparse: 0.0302394135811
	Values: 4 [nan, 'A', 'B', 'C']
Feature: v4
	Type: float64
	Sparse: 0.435580514516
	Range: -0.0 20.0
	Mean: 4.1451 1.1483
Feature: v5
	Type: float64
	Sparse: 0.42532867977
	Range: -0.0 20.0
	Mean: 8.7424 2.036
Feature: v6
	Type: float64
	Sparse: 0.43589541729
	Range: -0.0 20.0
	Mean: 2.4364 0.6
Feature: v7
	Type: float64
	Sparse: 0.43589541729
	Range: -0.0 20.0
	Mean: 2.4839 0.5894
Feature: v8
	Type: float64
	Sparse: 0.425284943274
	Range: -0.0 20.0
	Mean: 1.4966 2.783
Feature: v9
	Type: float64
	Sparse: 0.436061615976
	Range: -0.0 20.0
	Mean: 9.0319 1.9303
Feature: v10
	Type: float64
	Sparse: 0.000734773138793
	Range: -0.0 18.53
	Mean: 1.883 1.394
Feature: v11
	Type: floa

In [39]:
variances = X.select_dtypes(exclude=['object']).apply(lambda f: round(f.std(), 5))
variances.sort_values(ascending=False)[:50]

ID      65934.48736
v100        6.88830
v58         6.84124
v119        4.24012
v23         4.00370
v109        3.71200
v82         3.67804
v117        3.61800
v40         3.16918
v87         3.00005
v63         2.96731
v2          2.96168
v25         2.95685
v106        2.95410
v46         2.89583
v89         2.81290
v8          2.78300
v28         2.67794
v114        2.61322
v123        2.60470
v64         2.52559
v51         2.51038
v39         2.35815
v36         2.35563
v101        2.35467
v98         2.32586
v54         2.31448
v70         2.31446
v68         2.28992
v105        2.24731
v81         2.22364
v48         2.19616
v61         2.12098
v44         2.11089
v124        2.09941
v59         2.07167
v45         2.06451
v5          2.03602
v34         2.02518
v118        2.00210
v9          1.93026
v97         1.92407
v17         1.91150
v115        1.90148
v69         1.90050
v65         1.87811
v78         1.84392
v102        1.82404
v122        1.79598
v16         1.79119


In [82]:
# Pairwise correlation
corrs = X.select_dtypes(include=['float64']).corr()
np.fill_diagonal(corrs.values, 0)  # Ignore self-correlation

maxes = corrs.apply(lambda f: pd.Series({'correlation': f.max(), 'neighbour': f.idxmax()}), axis=1)
#maxes.sort_values(by='correlation', ascending=False)

print maxes[maxes['correlation'] > 0.9]

for c in range(1, 10):
    max_c = float(c) / 10
    print 'Pairs with correlation > ', max_c, ':', len(maxes[maxes['correlation'] > max_c])


      correlation neighbour
v8       0.975974       v63
v10      0.911819       v12
v11      0.955757       v53
v12      0.911819       v10
v13      0.924397      v104
v15      0.945970       v73
v17      0.992671       v76
v20      0.911062       v65
v25      0.982215       v46
v26      0.980747       v60
v29      0.957029       v96
v32      0.912667       v86
v33      0.964302       v83
v34      0.912380      v114
v41      0.950463       v96
v43      0.977874      v116
v46      0.991609       v63
v49      0.904423       v41
v53      0.955757       v11
v54      0.967925       v25
v55      0.943377       v33
v60      0.980747       v26
v63      0.991609       v46
v64      0.982626       v17
v65      0.911062       v20
v67      0.932559       v41
v73      0.945970       v15
v76      0.992671       v17
v77      0.920373       v29
v83      0.966238      v121
v86      0.912667       v32
v89      0.964606       v25
v92      0.977089       v95
v95      0.977089       v92
v96      0.957029   